# VR ASSIGNMENT 2 - Q2a - IMT2019092

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [15]:
#Loading Images and Plotting
query_img = cv.imread('../input/panorama/image1.jpg')
queryimg_gray = cv.cvtColor(query_img, cv.COLOR_RGB2GRAY)

train_img = cv.imread('../input/panorama/image2.jpg')
trainimg_gray = cv.cvtColor(train_img, cv.COLOR_RGB2GRAY)

b,g,r = cv.split(query_img)
rgb_qi = cv.merge([r,g,b]) 

b,g,r = cv.split(train_img)      
rgb_ti = cv.merge([r,g,b]) 
plt.figure(figsize=(15,8))
plt.subplot(121)
plt.imshow(rgb_qi, cmap="gray")
ax = plt.gca()
ax.set_xlabel("Query image", fontsize=15)

plt.subplot(122)
plt.imshow(rgb_ti, cmap="gray")
ax = plt.gca()
ax.set_xlabel("Train image (Image to be transformed)", fontsize=15)

plt.show()

# Feature Extraction using SIFT Detector

In [16]:
#Function to run SIFT
def sift_detect(image):
     descriptor = cv.SIFT_create()
     (kps, features) = descriptor.detectAndCompute(image, None)
     return (kps, features)

In [17]:
#Extracting key points and features
kps_qi, features_qi = sift_detect(query_img)
kps_ti, features_ti = sift_detect(train_img)

In [18]:
#Plotting Keypoints of images
plt.figure(figsize=(15,8))
plt.subplot(121)
plt.imshow(cv.drawKeypoints(queryimg_gray, kps_qi, 0, color=(0,255,0), flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS))
ax = plt.gca()
ax.set_xlabel("Keypoints of Query Image", fontsize=13)

plt.subplot(122)
plt.imshow(cv.drawKeypoints(trainimg_gray, kps_ti, 0, color=(0,255,0), flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS))
ax = plt.gca()
ax.set_xlabel("Keypoints of Training Image", fontsize=13)


plt.show()

# Feature Matching using Brute Force

In [19]:
matcher = cv.BFMatcher(cv.NORM_L2, crossCheck = True)
raw_matches = matcher.match(features_qi, features_ti, None)
matches = sorted(raw_matches, key=lambda x:x.distance)
len(matches)

In [20]:
plt.figure(figsize=(15,8))
matched_image = cv.drawMatches(rgb_qi, kps_qi, rgb_ti, kps_ti, matches[:100], None, 
                                    flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(matched_image)
plt.show()

# Homography estimation using RANSAC

In [21]:
kps_qi = np.float32([kp.pt for kp in kps_qi])
kps_ti = np.float32([kp.pt for kp in kps_ti])

reproj_thresold = 4
if len(matches) > 4:
    pts_qi = np.float32([kps_qi[m.queryIdx] for m in matches])
    pts_ti = np.float32([kps_ti[m.trainIdx] for m in matches])
    (H, status) = cv.findHomography(pts_ti, pts_qi, cv.RANSAC, reproj_thresold)

print(H)

# Perspective Warping

In [22]:
width = rgb_qi.shape[1] + rgb_ti.shape[1]
height = rgb_qi.shape[0] + rgb_ti.shape[0]
result = cv.warpPerspective(rgb_ti, H, (width, height))
result[0:rgb_qi.shape[0], 0:rgb_qi.shape[1]] = rgb_qi

plt.figure(figsize=(15,10))
plt.imshow(result)

plt.axis('off')
plt.show()

# Using Inbuilt Package

In [23]:
stitcher = cv.Stitcher_create()
(status, stiched) = stitcher.stitch([query_img, train_img])
if status==0:
    b,g,r = cv.split(stiched)
    rgb_stiched = cv.merge([r,g,b]) 
    plt.imshow(rgb_stiched)
    plt.plot()